# Creational Patterns

- Every language gives you a standard way of creating new objects
    - In Python, you just declare a class and use __init__ and __new__ to control the construction
    - In Java, you have the ability to make constructors public or private

- These patterns are useful for cases where the standard construction patterns are not sufficient

- Creational design patterns provide various object creation mechanisms, which increase flexibility and reuse of existing code.

## 1. Factory Method

### Big Picture

- Factory method is just a general name for separating the constructor of any object from the work it needs to do
- There are a few scenarios where can be applied (see Use Cases), but the point is that: I have some object A where the actual construction is handled somewhere else
    - Ideally, the class you want to apply this factory method should not have a constructor, which is possible in languages like Java
    - In Python, you can't really do directly (though there are a workarounds)

### Example

In [22]:
from abc import ABC, abstractmethod

class Tool(ABC):
    @abstractmethod
    def tool_operation(self) -> None:
        pass

class User(ABC):
    @abstractmethod
    def get_tool(self) -> Tool:
        pass

    def use_tool(self) -> str:
        ## Make Tool
        tool = self.get_tool()
        result = f" Creator {self.__class__.__name__} is using tool {tool.__class__.__name__}"
        tool.tool_operation()
        return result

class User1(User):
    def get_tool(self) -> Tool:
        print(' Creator has picked up Tool 1')
        return Tool1()

class Tool1(Tool):
    def tool_operation(self) -> None:
        print(" Tool 1 is doing something")

class User2(User):
    def get_tool(self) -> Tool:
        print(' Creator has picked up Tool 2')
        return Tool2()

class Tool2(Tool):
    def tool_operation(self) -> None:
        print(" Tool 2 is doing something")

def client_code(creator: User) -> None:
    print(f"Printing from client_code(): Client is not aware of which user has been passed to it, or the tool the user is using. And yet, this code still works: `creator.use_tool()`", end='\n')
    print(f"{creator.use_tool()}", end='\n')
    print(f"This works because the creator and the product both respect a common interface", end='\n')

print("App: Launched with the a concrete creator.")
client_code(User1())

print("\n" + "+"*50)

print("App: Launched with the other concrete creator.")
client_code(User2())

App: Launched with the a concrete creator.
Printing from client_code(): Client is not aware of which user has been passed to it, or the tool the user is using. And yet, this code still works: `creator.use_tool()`
 Creator has picked up Tool 1
 Tool 1 is doing something
 Creator User1 is using tool Tool1
This works because the creator and the product both respect a common interface

++++++++++++++++++++++++++++++++++++++++++++++++++
App: Launched with the other concrete creator.
Printing from client_code(): Client is not aware of which user has been passed to it, or the tool the user is using. And yet, this code still works: `creator.use_tool()`
 Creator has picked up Tool 2
 Tool 2 is doing something
 Creator User2 is using tool Tool2
This works because the creator and the product both respect a common interface


### Discussion

- There is some jargon needed to discuss a factory method
    - `Product` class: The class object that you want created
    - `Creator` class: The class responsible for creating the `Product`
        - The `Creator` may or may not use the created `Product` to do something
        - In the example above, the Creator class is `User()`, and the Product class is `Tool()`
    - `Client`: Whatever code is using the `Creator` and/or `Product` classes to do something
 
- From the example above, the whole point of the factory method approach is to write the code in a way that is not coupled with each other
    - i.e. `client_code()` will work regardless of what creator is passed to it, because all creators implement `use_tool()`
    - i.e. The creator will work with any `Tool`, because all tools implement `tool_operation()`

### Pros and Cons

- Pros
    - No tight coupling of objects, because the code will work so long as the object respects a common interface
    - Single responsibility: the product creation is in a single place, which makes the code easier to support
    - Open closed principle: You can introduce new products without breaking existing client code

- Cons
    - You may need to introduce a lot of classes to implement this pattern, which is not idael

### Use Cases

- You don't know what types and/or dependencies your code should work with
    - By separating the construction of dependencies into a specific component of your code, it makes things easier to change/extend 

- You want to allow users to extend your library/framework easily
    - Basically, anything that follows the interfaces defined will work with your library. So 3rd party users are free to write their own code that respects these interfaces, because the abstract methods can always be overwritten

- You want to save system resources by reusing existing objects
    - Suppose we are running an umbrella stand, where users can borrow and return umbrellas after they are done with it
    - We have multiple `Umbrella` objects
    - If someone comes to our stand to request an `Umbrella`, we want to check whether there are existing ones. If there are, we give him an existing one. If there isn't, we buy (instantiate) a new one, and loan it out
    - Obviously, you cannot (should not) track your inventory of umbrellas within the `Umbrella` class, because it will only (should only) know about itself
        - You can technically do this via class attributes, but it's really odd to have multiple copies of the `Umbrella` which are supposedly distinct, yet somehow share a common state tracking component
    - So you create an `UmbrellaStand` object, and introduce a constructor method for `Umbrella`
        - For example, you could put a field called `self.available_umbrellas = [Umbrella1, Umbrella2, ...]`
        - And make a method called `def get_new_umbrella(self): ...`
        - `if len(self.available_umbrellas) == 0: self.get_new_umbrella()`



## 2. Abstract Factory

### Big Picture

- There are occasions where you want to make a factory to replace constructors in individual objects (see `Factory Method`)
- But you also want these factories to follow have some common pattern, or it'll create compatibility issues
- So you define an interface (called an `Abstract Factory`) that standardises what all factories must have, and what they must be able to do
- Note: The abstract factory does NOT handle the construction of the concrete classes, only the blueprint for the factory making these classes

### Example

In [30]:
from abc import ABC, abstractmethod


class AbstractFactory(ABC):
    @abstractmethod
    def create_product_a(self) -> 'AbstractProductA':
        pass

    @abstractmethod
    def create_product_b(self) -> 'AbstractProductB':
        pass

class AbstractProductA(ABC):
    @abstractmethod
    def useful_function_a(self) -> str:
        pass

class AbstractProductB(ABC):
    @abstractmethod
    def useful_function_b(self) -> str:
        pass

    @abstractmethod
    def another_useful_function_b(self, collaborator: AbstractProductA) -> str:
        pass

class ConcreteFactory1(AbstractFactory):
    def create_product_a(self) -> AbstractProductA:
        return ConcreteProductA1()

    def create_product_b(self) -> AbstractProductB:
        return ConcreteProductB1()

class ConcreteFactory2(AbstractFactory):
    def create_product_a(self) -> AbstractProductA:
        return ConcreteProductA2()

    def create_product_b(self) -> AbstractProductB:
        return ConcreteProductB2()

class ConcreteProductA1(AbstractProductA):
    def useful_function_a(self) -> str:
        return "The result of the product A1."

class ConcreteProductA2(AbstractProductA):
    def useful_function_a(self) -> str:
        return "The result of the product A2."

class ConcreteProductB1(AbstractProductB):
    def useful_function_b(self) -> str:
        return "The result of the product B1."

    def another_useful_function_b(self, collaborator: AbstractProductA) -> str:
        result = collaborator.useful_function_a()
        return f"The result of the B1 collaborating with the ({result})"

class ConcreteProductB2(AbstractProductB):
    def useful_function_b(self) -> str:
        return "The result of the product B2."

    def another_useful_function_b(self, collaborator: AbstractProductA):
        result = collaborator.useful_function_a()
        return f"The result of the B2 collaborating with the ({result})"

def client_code(factory: AbstractFactory) -> None:
    product_a = factory.create_product_a()
    product_b = factory.create_product_b()

    print(f"{product_b.useful_function_b()}")
    print(f"{product_b.another_useful_function_b(product_a)}", end="")

print("Client: Testing client code with the first factory type:")
client_code(ConcreteFactory1())
print("\n")
print("Client: Testing the same client code with the second factory type:")
client_code(ConcreteFactory2())

Client: Testing client code with the first factory type:
The result of the product B1.
The result of the B1 collaborating with the (The result of the product A1.)

Client: Testing the same client code with the second factory type:
The result of the product B2.
The result of the B2 collaborating with the (The result of the product A2.)

### Discussion

- I think it's clear that this is just another level of indirection compared to the first section on `Factory Methods`
    - Similar to Factory Methods, instead of instantiating the products you want, you let a "factory" handle it e.g. `factory.create_product_a()`
    - But the difference here is that you go a step further, and even the factory you use is also abstracted, which lets you handle the creation of products via different factories dynamically also

### Pros and Cons

- Pros
    - The products from all your factories are definitely going to be compatiable with each other
    - No tight coupling between client code and concrete product getting created
    - Single responsibility: Only 1 place responsible for construction code
    - Open/Closed principple: YOu can introduce new product variants without breaking existing client code

- Cons
    - The code gets pretty complicated for no good reason, because a lot of interfaces and classes are introduced 

### Use Cases

- Your code needs to work with a few families of related products, but you don't know what the concrete classes of these products are beforehand
    - So instead of relying on the actual concrete products, you only rely on their creator!

- When you have a class with a set of Factory Methods, it may blur the responsibility of the class. Is it responsible for creating something? Doing something?
    - Use a abstract factory and create subclasses, so that the constructor and operation of the factory is clear

## 3. Builder

### Big Picture

- Imagine you want to make a House object

- There are many variations for houses: 
    - you can make a studio, 2-bedroom, etc...
    - it can have a balcony
    - it can have a garden
    - it can be 1/2/3/...floors
    - etc.

- The usual design is to pass all of this information as parameters into the house constructor
    - This is most likely going to be shit design
    - Because your parameter list will be huge, and the person using it will need to figure out which options can be null, which options should be a string, etc.

- Rather than front-load all the parameters, we create an object whose responsibility is to implement all the steps needed to build what you want to build. Let's call this the `Builder` object
    
- Though the `Builder` knows how each part of the building process is done, the `Builder` is also dumb, and does not know how to call them

- So to build something, we need another object to control the `Builder`, which we call the `Director`
    - The `Director` will be assigned a `Builder` 
    - Depending on what kind of house you want to build, you assign a Builder with the right knowledge
        - If you want to build a cottage, assign a `CottageBuilder`. 
        - If you want to build an apartment, assign a `ApartmentBuilder`. 
        - What's the difference? The CottageBuilder knows that a cottage must have a chimney :)

    - Strictly speaking a director isn't necessary; you can always just make the invocations to the `Builder` yourself!

### Example

In [ ]:
from abc import ABC, abstractmethod

class Builder(ABC):
    
    @abstractmethod
    def make_floor(self):
        ...

    @abstractmethod
    def make_roof(self):
        ...

class CottageBuilder(Builder):
    def __init__(self):
        ...
    
    def _lay_parquet(self):
        ...

    def _lay_tiles(self):
        ...

    def _make_chimney(self):
        ...

    def make_floor(self):
        self._lay_parquet()

    def make_roof(self):
        self._lay_tiles()
        self._make_chimney()

class ApartmentBuilder(Builder):
    def __init__(self):
        ...
    
    def _lay_concrete(self):
        ...

    def _attach_ceiling_fans(self):
        ...

    def _make_chimney(self):
        ...

    def make_floor(self):
        self._lay_concrete()

    def make_roof(self):
        self._attach_ceiling_fans()


cb = CottageBuilder()
cb.make_floor()
cb.make_roof()

### Discussion

- This pattern is most applicable when you have client code that will give specific instructions for the creation of something, and you want to give clients the flexibility of choosing what they want to achieve

### Pros and Cons

- Pros
    - You can construct objects step-by-step, defer construction steps or run steps recursively.
    - You can re-use the same construction vode when building multiple representations of the same product. In our example, the basic steps of making a house (floor, then roof), are common
    - Single Responsibility Principle: You can isolate complex construction logic away from the business logic 

- Cons
    - Your code base is more complex, because you now need to create a few new classes

### Use Cases

- You want your code to be able to create multiple representations of some product (e.g. stone or wood houses)
    - In the example above, slightly different things can be made in many different ways
    - So the `Builder` class is useful, so that each way is defined according to some strategy

- You want your code to construct complex objects
    - In some complex cases, you may want your object to have some funny nested behaviour that requires, say, recursive calls
    - For such cases, making such constructions by hand in the code may be difficult.
    - So it is much better to delegate the responsibility of the construction to a `Builder`


## 4. Prototype

### Big Picture

- When you have a large library to deal with, sometimes working with pre-built objects that you can modify is much easier
- However, it is often not easy to copy an object:
    - Often, you can only see the publicly available fields, so you may not be able to copy private attributes/methods
    - You need to know the object's class to create a duplicate, which creates a dependency on that class
    - You may only know the interface the object follows, but not the concrete class

- The `Prototype` pattern is just a fancy way of saying, we write an object that lets clients clone it at will

### Example

In [57]:
class Test:
    def __init__(self):
        self.a1 = 123
        self.a2 = 234
        # print(self.__dict__)

{'a1': 123, 'a2': 234}


In [63]:
import copy

class CircularReferenceObject:
    '''
    We'll use this object to make a circular reference in copying
    '''
    def __init__(self):
        self._parent = None

    @property
    def parent(self):
        return self._parent
    
    @parent.setter
    def parent(self, value):
        self._parent = value

class PrototypeObject:
    '''
    This is the object where we'll enable cloning
    '''
    def __init__(self, list_of_attrs, circular_reference_object):
        self.list_of_attrs = list_of_attrs
        self.circular_reference_object = circular_reference_object

    def __copy__(self):
        copy_list_of_attrs = copy.copy(self.list_of_attrs)
        copy_circular_reference_object = copy.copy(self.circular_reference_object)
        new = self.__class__(
            copy_list_of_attrs, copy_circular_reference_object
        )
        new.__dict__.update(self.__dict__)
        return new
    
    def __deepcopy__(self, memo=None):
        if memo is None:
            memo = {}
        copy_list_of_attrs = copy.deepcopy(self.list_of_attrs, memo)
        copy_circular_reference_object = copy.deepcopy(self.circular_reference_object, memo)
        new = self.__class__(
            copy_list_of_attrs, copy_circular_reference_object
        )
        new.__dict__ = copy.deepcopy(self.__dict__, memo)
        return new

list_of_attrs = [1, {1, 2, 3}, [1, 2, 3]]
circular_ref = CircularReferenceObject()
prototype = PrototypeObject(list_of_attrs, circular_ref)
circular_ref.parent = prototype
shallow_copied_prototype = copy.copy(prototype)
## Shallow copies are affected by amendments to the original object
shallow_copied_prototype.list_of_attrs.append('Appending to shallow copy')
print(prototype.list_of_attrs)

## If we implement deepcopies for the Prototype object, we get the exact object but independent of the original
prototype = PrototypeObject(list_of_attrs, circular_ref)
circular_ref.parent = prototype
deep_copied_prototype = copy.deepcopy(prototype)
deep_copied_prototype.list_of_attrs.append('Appending to deep copy')
print(prototype.list_of_attrs)
print(deep_copied_prototype.list_of_attrs)

[1, {1, 2, 3}, [1, 2, 3], 'Appending to shallow copy']
[1, {1, 2, 3}, [1, 2, 3], 'Appending to shallow copy']
[1, {1, 2, 3}, [1, 2, 3], 'Appending to shallow copy', 'Appending to deep copy']


In [53]:
# id(shallow_copied_component.some_circular_ref.parent.some_circular_ref.parent.some_circular_ref)

### Discussion

- Why do I want to do this?
    - The construction of an object can be expensive, so cloning lets you avoid that cost
    - Cloning lets you preserve the state of an object, whereas initialisation does not
    - Cloning lets you isolate the changes made to an object from whatever state it is in
    - Cloning can be better for concurrent/multithreaded code, because you can easily create exact copies of the same object that are also independent

### Pros and Cons

- Pros
    - You can clone objects without coupling to their concrete classes.
    - You can get rid of repeated initialization code in favor of cloning pre-built prototypes.
    - You can produce complex objects more conveniently.
    - You get an alternative to inheritance when dealing with configuration presets for complex objects.

- Cons
    - Cloning complex objects that have circular references might be very tricky

### Use Cases

- Use the Prototype pattern when your code shouldn’t depend on the concrete classes of objects that you need to copy.

- Use the pattern when you want to reduce the number of subclasses that only differ in the way they initialize their respective objects.
    - Prototypes lets you avoid cluttering up your code when you have multiple ways of constructing some object, but you don't want to have a million subclasses in your code base

## 5. Singleton in Python

- **Singletons are considered to be an anti-pattern, so please try not to use it**

- Why it is bad
    - Tight coupling of your code to the singleton

- Best way to think through how the coupling can break
    - Think about what happens to your code when you multi-thread it
    - The best way to do this, is to imagine unit testing your code and running multiple tests at once

### Big Picture

- Singletons are a design pattern where you ensure one and only one object of its kind exists, and there is a single point of access to it for all other code
    - A good use case for Singletons is a database connection

### Example

#### Naive Singleton (no multithread)

- In this cell block, we define a metaclass to check whether the instance has been created before

In [93]:
class SingletonMeta(type):
    _instance_store = {}

    def __call__(cls, *args, **kwargs):
        if cls not in cls._instance_store:
            print(f'Creating new instance of {cls.__name__}')
            new_cls = super().__call__(*args, **kwargs)
            cls._instance_store[cls] = new_cls
        return cls._instance_store[cls]
    
class SingletonNaive(metaclass=SingletonMeta):
    def some_business_logic(self):
        print('doing something')

class DifferentSingletonNaive(metaclass=SingletonMeta):
    def some_business_logic(self):
        print('doing something')

s1 = SingletonNaive()
s2 = SingletonNaive()
s3 = DifferentSingletonNaive()

print(f"Repeated objects do not get recreated because memory IDs are the same: {id(s1)=}, {id(s2)=}")

Creating new instance of SingletonNaive
Creating new instance of DifferentSingletonNaive
Repeated objects do not get recreated because memory IDs are the same: id(s1)=4439943984, id(s2)=4439943984


- However, there are other ways of enforcing Singleton creation without having to create a Metaclass! 

- For example, we can use the `__new__` dunder method in classes too

- Upside is, we can make the singleton behaviour without a new metaclass

- Downside is, you can't reuse the Singleton construction as simply, or as cleanly

In [94]:
class SingletonNaive():
    _existing_instance = False

    def __new__(cls, *args, **kwargs):
        if not cls._existing_instance:
            cls._existing_instance = super().__new__(cls, *args, **kwargs)
        return cls._existing_instance
            
    def some_business_logic(self):
        print('doing something')


s1 = SingletonNaive()
s2 = SingletonNaive()
print(f"Repeated objects do not get recreated because memory IDs are the same: {id(s1)=}, {id(s2)=}")

Repeated objects do not get recreated because memory IDs are the same: id(s1)=4439941680, id(s2)=4439941680


#### Thread-safe Singleton

- Thread safety is also quite easily enforced, just lock the object while it is being used

In [102]:
from threading import Lock, Thread

class SingletonMeta(type):
    _instance_store = {}
    _lock = Lock()

    def __call__(cls, *args, **kwargs):
        with cls._lock:
            if cls not in cls._instance_store:
                instance = super().__call__(*args, **kwargs)
                cls._instance_store[cls] = instance
        return cls._instance_store[cls]
    
class SingletonThreadSafe(metaclass=SingletonMeta):
    def __init__(self, attr):
        self.attr = attr

    def some_business_logic(self):
        print('doing something')

def init_singleton_with_value(value):
    s = SingletonThreadSafe(value)
    print(s.attr, end='\n')

process1 = Thread(target=init_singleton_with_value, args=('VALUE1',))
process2 = Thread(target=init_singleton_with_value, args=('VALUE2',))
process3 = Thread(target=init_singleton_with_value, args=('VALUE3',))
process1.start()
process2.start()
process3.start()
print('When multiple threads try to create a singleton with a different attribute value, it is unable to because the object cannot be operated on until the first process is complete (i.e. Locked)')

VALUE1
VALUE1
VALUE1
When multiple threads try to create a singleton with a different attribute value, it is unable to because the object cannot be operated on until the first process is complete (i.e. Locked)


### Discussion

- Singletons break modularity of code
    - You can’t really re-use a class that depends on a Singleton in another context, unless that Singleton is also ported over. 
    - So basically the Singleton class becomes coupled to everything else

- Singleton pattern breaks single responsibility principle (though I'd say it's probably not a huge deal)
    - They ensure the class has one and only one instance
    - They provide a way for code to access this object
        - Singletons usually have some method that allows clients to access the singleton (if it has been constructed), or to make one (if it hasn't been constructed)

### Pros and Cons

- Pros
    - You can guarantee that there is only a single instance of a class
    - Any part of your code can access the singleton through a single access point
    - Especially in cases where the initialisation is expensive, singletons are useful because you only incur this cost once

- Cons
    - Violates the Single Responsibility Principle. The pattern solves two problems at the time (ensures the class has only 1 instance, AND provides a way to access the object)
    - The Singleton pattern can mask bad design, for instance, when the components of the program know too much about each other.
        - i.e. This introduces very very strong coupling, so you should be careful about how this 
    - You HAVE to implement some special treatment if you are dealing with multithreaded applications, because withoutn this, multiple threads can create a singleton object several times.
    - Unit testing of singletons is hard, because most unit test libraries rely on inheritance when producing mock objects, and the constructor class is private 

### Use Cases

- You have an object that is SUPER expensive to initialise, and you really only need one of it
    - So you disable any new object from getting created once you incur this cost once

- You need to control the number of objects you have strictly
    - e.g. your singleton controls access to some resource that supports a maximum of 5 concurrent users. So you want a maximum of 5 connections to be created